# Unsupervised Learning: Von Der Malsburg (VDM) model of V1 Orientation Columns

# Overview
This Jupyter Notebook implements the VDM model [[1]](#1) of a layer of a V1 neural patch that takes line orientation inputs from a fixed location in the visual space. After training, each neural unit in the V1 patch activity exhibits selective response to a specific orientation and the patch exhibits a spatial gradient manifold of differential orientation selectivities across the surface of the patch (observed in Tree Shrews [[2]](#2)). This mimics the primary findings from Hubel & Wiesel in cat and monkey lateral geniculate and visual regions [[3]](#3)[[4]](#4).

# Define the Problem

# Getting an artificial neural network to show self-organization for clusters and gradients of orientation selectivity across columns.
- Functional activity of neurons in the brain exhibit some organizational patterns (e.g., modules, local clusters, topology, etc.) that are not easily explained by biological determinism (i.e., genetics).
- Thus, an interesting question emerges: If the genetic (and epigenetic) mechanisms are insufficient to account for brain neural organization, what mechanism is in operation?
- Von Der Malsburg addresses this question by examining the formation of self-organized column cluster gradients of orientation selectivity that is observed to occur in V1 of certain animals (e.g., shrews, cats, and monkeys).
- The approach used is to implement a neural network architecture that is based on known V1 connectivity. Essentially, a layer of a patch of V1 neurons receives bottom up input from lower retinal sources (essentially), with within-layer excitatory connections (from other V1 neurons) as well as between-layer inhibitor connections (from GABA-ergic inter-neurons).
- Based on this architecture, standard learning rules (auto-encoding), and activation functions (regularized linear unit, ReLU), we aim to evaluate simulated V1 neural activies and see if they show self-organization of columns into clusters with gradients of orientation selectivity.

**References**
1. <a id="1"></a>Von Der Malsburg, C. (1973). Self-organization of orientation sensitive cells in the striate cortex. Kybernetik, 14, 85-100. [[pdf]](https://cool.ntu.edu.tw/courses/45064)
2. <a id="2"></a>Bosking, W. H., Zhang, Y., Schofield, B., & Fitzpatrick, D. (1997). Orientation selectivity and the arrangement of horizontal connections in tree shrew striate cortex. The Journal of Neuroscience, 17(6), 2112–2127. https://doi.org/10.1523/JNEUROSCI.17-06-02112.1997
3. <a id="3"></a>Wiesel, T. N., & Hubel, D. H. (1963). Effects of visual deprivation on morphology and physiology of cells in the cats lateral geniculate body. Journal of Neurophysiology, 26, 978–993. https://doi.org/10.1152/jn.1963.26.6.978
4. <a id="4"></a>Hubel, D. H., & Wiesel, T. N. (1974). Sequence regularity and geometry of orientation columns in the monkey striate cortex. The Journal of Comparative Neurology, 158(3), 267–293. https://doi.org/10.1002/cne.901580304
5. Goh, J. O. S. (2007). Extensions of the von der Malsburg self-organized visual cortex model. Unpublished. https:/gitlab.com/joshuagoh/unsupervised-learning. [PDF](https://gitlab.com/joshuagoh/unsupervised-learning/-/blob/main/manuscript/VDM-joshgoh.pdf)

# Data Table
- A key distinction in the VDM implementation is that the unit positions of the input vectors or arrays are not arbitrary.
- The relative position of a given input unit with respect to other input unit has meaning. i.e., inter-unit distances have meaning.
- The approach that applies minimal spatial distance biases of a given point to all other radial points is the circle and its circumference around which all positions are equidistant to the centroid.
- For digital formats, the centroid and vertices of a hexagon is a reasonable approximations for a circle.
- We thus first have to devise a base transform function to work with hexagonal node arrays (given vector input states), and corresponding visualizations.

## Import modules

In [1]:
# Modules
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

## Define function for transforming between coordinates of a hexagonal grid manifold and the Cartesian manifold.

Consult: https://www.redblobgames.com/grids/hexagons/

In [2]:
def create_hex_grid(rows, cols, spacing):
    """
    Creates a hexagonal grid of points.

    Args:
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        spacing (float): The spacing between hexagon centers.

    Returns:
        tuple: A tuple containing two numpy arrays, x and y coordinates of the grid points.
    """
    x_coords = []
    y_coords = []

    h = spacing/2 * np.tan(np.deg2rad(60))

    for r in range(rows):
        for c in range(cols):
            x = c * spacing/2
            y = r * 2 * h
            if c % 2 != 0: # offset odd columns
                y += h

            x_coords.append(x)
            y_coords.append(y)

    return np.array(x_coords), np.array(y_coords)

## Define function to visualize VDM hexagonal input ON states.

In [3]:
def vdmhexplot(x,y,onlist,subplot_nrows,subplot_ncols,subplot_titles,plot_title,Cart_grid_on,Tick_labels_on):

  fig = make_subplots(
    rows=subplot_nrows,
    cols=subplot_ncols,
    subplot_titles=subplot_titles,
    )

  plot_idx = 0
  for r in range(1, subplot_nrows + 1):
    for c in range(1, subplot_ncols + 1):

      # Create the Plotly figure
      ## Units that are ON
      fig.add_trace(go.Scatter(
        x=x[np.where(onlist[plot_idx]==1)[0]],
        y=y[np.where(onlist[plot_idx]==1)[0]],
        mode='markers',
        marker=dict(
          size=10,
          color='black',
          symbol='circle')
        ),row=r, col=c)

      ## Units that are OFF
      fig.add_trace(go.Scatter(
        x=x[np.where(onlist[plot_idx]==0)[0]],
        y=y[np.where(onlist[plot_idx]==0)[0]],
        mode='markers',
        marker=dict(
          size=3,
          color='black',
          symbol='circle')
        ),row=r, col=c)
      plot_idx += 1

  # Update layout for better visualization
  fig.update_layout(
    width=500,
    height=500,
    title=plot_title,
    showlegend=False,
    template='plotly_white'
  )

  #fig.update_yaxes(scaleanchor="x", scaleratio=4,showgrid=Cart_grid_on,showticklabels=Tick_labels_on,zeroline=False)
  fig.update_yaxes(showgrid=Cart_grid_on,showticklabels=Tick_labels_on,zeroline=False)
  fig.update_xaxes(showgrid=Cart_grid_on,showticklabels=Tick_labels_on,zeroline=False)

  # Show the plot
  fig.show()

## Visualize manifolds

In [4]:
# Define grid dimensions and unit sub-sets
grid_rows = 8
grid_cols = 29
spacing = 1.0 # radius
onlist = [np.ones(grid_rows*grid_cols)]
subplot_nrows = 1
subplot_ncols = 1
subplot_titles = ['']
plot_title = 'Hexagon in Cartesian Manifold'
Cart_grid_on = True
Tick_labels_on = True

# Get hexagonal grid manifold coordinates
x, y = create_hex_grid(grid_rows, grid_cols,spacing)

# Plot stimuli
vdmhexplot(x,y,onlist,subplot_nrows,subplot_ncols,subplot_titles,plot_title,Cart_grid_on,Tick_labels_on)

## Visualize stimuli

In [5]:
# Define grid dimensions and unit sub-sets
grid_rows = 3
grid_cols = 9
spacing = 1.0 # radius
sel = np.array([2,4,6,1,3,5,7,9,11,13,15,17,10,12,14,16,20,22,24])
onlist = np.array([[0,1,0,0,1,1,0,0,0,1,0,0,0,1,1,0,0,1,0],
                   [1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1],
                   [1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1],
                   [0,0,0,1,1,0,0,0,1,1,1,0,0,0,1,1,0,0,0],
                   [0,0,0,1,0,0,0,1,1,1,1,1,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0],
                   [0,0,0,0,0,1,1,0,1,1,1,0,1,1,0,0,0,0,0],
                   [0,0,1,0,0,1,1,0,0,1,0,0,1,1,0,0,1,0,0],
                   [0,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0]])
subplot_nrows = 3
subplot_ncols = 3
subplot_titles = ['1','2','3','4','5','6','7','8','9']
plot_title = 'Input Unit State Patterns'
Cart_grid_on = False
Tick_labels_on = False

# Get hexagonal grid manifold coordinates
x, y = create_hex_grid(grid_rows, grid_cols,spacing)

# Plot stimuli
vdmhexplot(x[sel],y[sel],onlist,subplot_nrows,subplot_ncols,subplot_titles,plot_title,Cart_grid_on,Tick_labels_on)

## Visualize one cortical layer

In [6]:
# Define grid dimensions and unit sub-sets
grid_rows = 8
grid_cols = 29
spacing = 1.0
sel = np.array([7,9,11,13,15,17,19,21,
                35,37,39,41,43,45,47,49,51,
                34,36,38,40,42,44,46,48,50,52,
                62,64,66,68,70,72,74,76,78,80,82,
                61,63,65,67,69,71,73,75,77,79,81,83,
                89,91,93,95,97,99,101,103,105,107,109,111,113,
                88,90,92,94,96,98,100,102,104,106,108,110,112,114,
                116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,
                117,119,121,123,125,127,129,131,133,135,137,139,141,143,
                147,149,151,153,155,157,159,161,163,165,167,169,171,
                148,150,152,154,156,158,160,162,164,166,168,170,
                178,180,182,184,186,188,190,192,194,196,198,
                179,181,183,185,187,189,191,193,195,197,
                209,211,213,215,217,219,221,223,225,
                210,212,214,216,218,220,222,224])
onlist = [np.ones(np.shape(sel)[0])]
subplot_nrows = 1
subplot_ncols = 1
subplot_titles = ['E-Layer']
plot_title = 'Cortical Layer'
Cart_grid_on = False
Tick_labels_on = False

# Get hexagonal grid manifold coordinates
x, y = create_hex_grid(grid_rows, grid_cols,spacing)

# Plot stimuli
vdmhexplot(x[sel],y[sel],onlist,subplot_nrows,subplot_ncols,subplot_titles,plot_title,Cart_grid_on,Tick_labels_on)

## Define function to train VDM network

In [7]:
def vdmulearning(W,p,r,q,s,h,t,order,input,theta):

  NUM_STABILIZATION_ITERATIONS = 20 # Hard parameter

  # Setup weights
  # For untrained weights
  if not(isinstance(W, np.ndarray)): # If no trained W array, set W to be be the length of input vector, and row and col sizes of the cortical layer (3 scalars).

    # General parameters
    ni = W[0]
    eesep = order[0]
    eisep = order[1]
    iesep = order[2]

    # Set cortical-layer hexagonal manifold coordinates and indices
    grid_rows = W[1]
    grid_cols = W[2]
    spacing = 1.0 # radius, or min distance between two units
    x, y = create_hex_grid(grid_rows, grid_cols,spacing)
    sel = np.array([7,9,11,13,15,17,19,21,
                35,37,39,41,43,45,47,49,51,
                34,36,38,40,42,44,46,48,50,52,
                62,64,66,68,70,72,74,76,78,80,82,
                61,63,65,67,69,71,73,75,77,79,81,83,
                89,91,93,95,97,99,101,103,105,107,109,111,113,
                88,90,92,94,96,98,100,102,104,106,108,110,112,114,
                116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,
                117,119,121,123,125,127,129,131,133,135,137,139,141,143,
                147,149,151,153,155,157,159,161,163,165,167,169,171,
                148,150,152,154,156,158,160,162,164,166,168,170,
                178,180,182,184,186,188,190,192,194,196,198,
                179,181,183,185,187,189,191,193,195,197,
                209,211,213,215,217,219,221,223,225,
                210,212,214,216,218,220,222,224])

    # Initialize input, excitatory, and inhibitory layer connection weights
    nE = np.shape(sel)[0]
    nI = nE
    WiE = np.random.uniform(0,s,(ni,nE))  # input-Excitatory
    WEE = np.zeros((nE,nE))               # Excitatory-Excitatory
    WEI = WEE.copy()                      # Excitatory-Inhibitory
    WIE = WEE.copy()                      # Inhibitory-Excitatory
    WEEd = np.zeros((nE,nE))              # Cortical inter-unit distances

    # Apply "genetically determined" weight hyperparameters
    for u1 in range(nE):
      for u2 in range(nE):
        dy = y[sel[u2]] - y[sel[u1]]
        dx = x[sel[u2]] - x[sel[u1]]

        WEEd[u1,u2]=np.sqrt(dx**2+dy**2)

        # E-E
        if (round(np.sqrt(dx**2+dy**2))==round(eesep*spacing)):
          WEE[u1,u2] = p

        # E-I
        if (round(np.sqrt(dx**2+dy**2))<=round(eisep*spacing)):
          WEI[u1,u2] = r

        # I-E
        if (round(np.sqrt(dx**2+dy**2))==round(iesep*spacing)):
          WIE[u1,u2] = q

  # For pre-trained weights
  else:
    WiE = W[0].copy()
    WEE = W[1].copy()
    WEI = W[2].copy()
    WIE = W[3].copy()

    # Set parameters
    ni = np.shape(WiE)[0]
    nE = np.shape(WiE)[1]
    nI = nE

  # Train network
  # Initialize training parameters
  asEt = []   # excitatory states over runs
  asIt = []   # inhibitory states over runs
  asEdt = []  # excitatory states over iterations
  asIdt = []  # inhibitory states over iterations
  sigE = np.empty(nE)   # excitatory output signals
  sigI = np.empty(nI)   # inhibitory output signals
  WiEt = []   # input-E weights over runs
  WEEt = []   # E-E weights over runs
  WEIt = []   # E-I weights over runs
  WIEt = []   # I-E weights over runs
  maxast = [] # Max. unit difference between two iteractions
  ip = []
  excite = []
  inhib = []

  # Start runs
  for run in range(t):

    asEp = []  # Excitatory states for all patterns in a run
    asIp = []  # Inhibitory states for all patterns in a run

    # Normalize weights                                             ## CHECK
    sumsk = np.sum(WiE,axis=0) + 1e-9
    preWiE = WiE.copy()
    WiE = preWiE * np.full((ni, nE), (ni * s) / (2 * sumsk))
    WiEp = WiE.copy()*0

    # Loop through input patterns
    for pat in range(np.shape(input)[0]):
      inputp = input[pat]
      asE = np.zeros((nE,1))
      asI = np.zeros((nI,1))

      # Begin stabilizing network based on input
      for count in range(NUM_STABILIZATION_ITERATIONS):

        # Log activation states
        asEdt.append(asE.copy())
        asIdt.append(asI.copy())

        # Compute excitatory output signals
        for j in range(nE):
          sigE[j]=max(asE[j]-theta,0)

        # Compute inhibitory output signals
        asI = np.dot(WEI.T,sigE)
        for j in range(nI):
          sigI[j]=max(asI[j]-theta,0)

        # Compute excitatory activation states
        ip.append(np.dot(WiE.T,inputp))
        excite.append(np.dot(WEE.T,sigE))
        inhib.append(np.dot(WIE.T,sigI))
        asEnew = np.dot(WiE.T,inputp) + np.dot(WEE.T,sigE) - np.dot(WIE.T,sigI)
        maxast.append(np.max(asEnew-asE))
        asE = asEnew.copy()

      # Log activation states
      asEdt.append(asE.copy())
      asIdt.append(asI.copy())

      # Update weights and unit activation states for each pattern
      asEp.append(asE.copy())
      asIp.append(asI.copy())
      WiEp = WiEp + h*np.outer(inputp,sigE) # Learning (updating) rule

    # Update weights (batch updating)
    WiE = WiE.copy() + WiEp

    # Update variables over runs
    WiEt.append(WiE.copy())
    WEEt.append(WEE.copy())
    WEIt.append(WEI.copy())
    WIEt.append(WIE.copy())
    asEt.append(asEp.copy())
    asIt.append(asIp.copy())

  return WEEd, WiEt, WEEt, WEIt, WIEt, asEt, asIt, asEdt, asIdt, maxast, ip, excite, inhib


# Simulation 1

In [8]:
# Input
input = np.array([[0,1,0,0,1,1,0,0,0,1,0,0,0,1,1,0,0,1,0],
                   [1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1],
                   [1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1],
                   [0,0,0,1,1,0,0,0,1,1,1,0,0,0,1,1,0,0,0],
                   [0,0,0,1,0,0,0,1,1,1,1,1,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0],
                   [0,0,0,0,0,1,1,0,1,1,1,0,1,1,0,0,0,0,0],
                   [0,0,1,0,0,1,1,0,0,1,0,0,1,1,0,0,1,0,0],
                   [0,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0]])

# Hyperparameters
W=[19,8,29]
p=0.4           # Strength of excitatory-excitatory weights
q=0.3           # Strength of excitatory-inhibitory weights
r=0.286         # Strength of inhibitory-excitatory weights
s=0.25          # Maximum initial strength of input-excitatory weight
h=0.05          # Growth rate
theta=1         # Activation function threshold
order=[1,1,2]   # List of orders (unit radius) of influences for cortical layer weights
                #  - 1: E-E
                #  - 2: E-I
                #  - 3: I-E
t = 100         # Number of runs

In [9]:
WEEd,WiEt,WEEt,WEIt,WIEt,asEt,asIt,asEdt,asIdt,maxast,ip,excite,inhib = vdmulearning(W,p,r,q,s,h,t,order,input,theta)

In [10]:
# prompt: Use plotly. Visualize W's as a heatmap.

fig = go.Figure(data=go.Heatmap(
    z=WIEt[-1],
    colorscale='Viridis'
))

fig.update_layout(
    title='Heatmap',
    xaxis_title='Excitatory Unit Index',
    yaxis_title='Excitatory Unit Index',
    width=500,
    height=500
)

fig.show()

In [20]:
# @title Evaluate Excitatory Layer {"run":"auto"}
input_state = 5 # @param {"type":"slider","min":0,"max":8,"step":1}
iteration = 0 # @param {"type":"slider","min":0,"max":99,"step":1}

# Binarize excitatory layer for plotting
onlist[0][np.where(asEt[iteration][input_state]>theta)]=1
onlist[0][np.where(asEt[iteration][input_state]<=theta)]=0

# Plot excitatory layer
print('Iteration: '+str(iteration))
print('Input State: '+str(input_state))
vdmhexplot(x[sel],y[sel],onlist,subplot_nrows,subplot_ncols,subplot_titles,plot_title,Cart_grid_on,Tick_labels_on)

Iteration: 0
Input State: 5
